In [1]:
%load_ext autoreload
%autoreload 2
    
%load_ext tensorboard
import sys, os
sys.path.append('../paraphrase/')
from paraphraser_args import ModelArguments, DataTrainingArguments, TrainingArguments
from paraphraser_dataloader import load_dataset_pseudo
from paraphraser_trainer import ParaphraserTrainer
from transformers import AutoTokenizer, AutoModelWithLMHead, HfArgumentParser

In [2]:
data_dir = "../data/pseudo"
task = "formality"
model_name = "t5-small"
model_nick = "t5_transfer_diff"
output_dir = "../models/"
epochs = "5"
train_batch_size = "16"
eval_batch_size = "16"
save_log_steps = "800"

parser = HfArgumentParser((ModelArguments, DataTrainingArguments, TrainingArguments))
model_args, data_args, training_args = parser.parse_args_into_dataclasses([
    "--model_name_or_path",
    model_name,
    "--model_nick",
    model_nick,
    "--data_dir",
    data_dir,
    "--output_dir",
    os.path.join(output_dir, model_nick),
    "--cache_dir",
    os.path.join(output_dir,"cache"),
    "--overwrite_cache",
    "--per_device_train_batch_size",
    train_batch_size,
    "--per_device_eval_batch_size",
    eval_batch_size,
    "--max_seq_len",
    "64",
    "--gradient_accumulation_steps",
    "1",
    "--num_train_epochs",
    epochs,
    "--logging_steps",
    save_log_steps,
    "--save_steps",
    save_log_steps,
    "--data_parallel",
    "True",
    "--meta_task",
    "transfer"
])


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_args.model_name_or_path)
model = AutoModelWithLMHead.from_pretrained(model_args.model_name_or_path)

loading configuration file https://huggingface.co/t5-small/resolve/main/config.json from cache at /home/vivek/.cache/huggingface/transformers/fe501e8fd6425b8ec93df37767fcce78ce626e34cc5edc859c662350cf712e41.406701565c0afd9899544c1cb8b93185a76f00b31e5ce7f6e18bbaef02241985
Model config T5Config {
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
    

In [4]:
train_dataset = load_dataset_pseudo(data_args.data_dir, tokenizer, mode="train", tasks = task.split('+'), n_proc=2048)
dev_dataset = load_dataset_pseudo(data_args.data_dir, tokenizer, mode="dev",  tasks = task.split('+'), n_proc=2048)

100%|██████████| 1/1 [00:00<00:00, 29.79it/s]


In [5]:
tokenizer.decode(train_dataset[0][0], skip_special_tokens=True), tokenizer.decode(train_dataset[0][2], skip_special_tokens=True)

('transfer: wut do i do in a nice way to make him stop? | input formality: low | output formality: high',
 'do you want to stop him in a good way?')

In [6]:
trainer = ParaphraserTrainer([training_args,model_args, data_args], model, tokenizer, train_dataset, dev_dataset)
trainer.train()

***** Running training *****
Num examples = 500
Num Epochs = 3.0
Total train batch size = 32
Gradient Accumulation steps = 1
Total optimization steps = 48.0
Logging steps = 20
Save steps = 20


***** Running Evaluation *****
Num examples = 100
Total eval batch size = 32



Configuration saved in ../models/t5_small/config.json
Model weights saved in ../models/t5_small/pytorch_model.bin
Saving model checkpoint to ../models/t5_small
New best model saved at step 20, epoch 1: score = 1.7997




KeyboardInterrupt: 

In [ ]:
training_args.meta_task

In [3]:
# Eval
tokenizer = AutoTokenizer.from_pretrained(model_args.model_name_or_path)
model = AutoModelWithLMHead.from_pretrained(os.path.join(output_dir, model_nick))

loading configuration file https://huggingface.co/t5-small/resolve/main/config.json from cache at /home/vivek/.cache/huggingface/transformers/fe501e8fd6425b8ec93df37767fcce78ce626e34cc5edc859c662350cf712e41.406701565c0afd9899544c1cb8b93185a76f00b31e5ce7f6e18bbaef02241985
Model config T5Config {
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
    

In [55]:
# example = "transfer: lol, u really think so? | input formality: low | output formality: high </s>"
example = "transfer: am i gonna get in trouble? | input to output: high"
# example = "transfer: did you hear about his problems, lol | input formality: low | output formality: high"
t = tokenizer(example, return_tensors='pt')

In [56]:
gen = model.generate(input_ids= t.input_ids, attention_mask = t.attention_mask, num_return_sequences = 3, num_beams = 12)

In [57]:
tokenizer.batch_decode(gen, skip_special_tokens=True)

['am I going to be in trouble?',
 'am I going to have trouble?',
 'do you have a problem with me?']